In [2]:
import requests
import pandas as pd
import csv
import re
import requests
from bs4 import BeautifulSoup
import urllib.request
import datetime
import json

# Get League Predicitons

In [2]:
df = pd.read_csv('./football_data/spi_matches_latest.csv')
df = df.drop(
        df.index[df['date'] == 'date'])
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [21]:
df = df.drop_duplicates()
df['probd'] = 1 - pd.to_numeric(
    df['prob1']) - pd.to_numeric(df['prob2'])
df['probd'] = df['probd'].round(2)

In [22]:
df = df[df['league'] == 'UEFA Champions League']
df

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2,probd
1504,2021,2021-09-14,1818,UEFA Champions League,Young Boys,Manchester United,65.12,85.6,0.198,0.5777,...,55.6,2,1,1.46,0.64,1.29,0.59,1.68,1.05,0.22
1505,2021,2021-09-14,1818,UEFA Champions League,Sevilla FC,FC Salzburg,80.64,78.05,0.4886,0.2709,...,78.9,1,1,1.47,2.79,1.16,0.99,1.05,1.05,0.24
1516,2021,2021-09-14,1818,UEFA Champions League,Barcelona,Bayern Munich,87.62,91.37,0.3796,0.412,...,43.6,0,3,0.23,2.56,0.41,2.34,0.0,2.77,0.21
1517,2021,2021-09-14,1818,UEFA Champions League,Lille,VfL Wolfsburg,67.94,77.05,0.3056,0.4027,...,75.0,0,0,1.37,0.06,1.83,0.39,0.0,0.0,0.29
1518,2021,2021-09-14,1818,UEFA Champions League,Malmo FF,Juventus,48.42,79.07,0.1318,0.6841,...,62.5,0,3,0.46,2.91,0.87,1.7,0.0,3.15,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9077,2021,2022-04-26,1818,UEFA Champions League,Manchester City,Real Madrid,93.35,85.58,0.6058,0.1815,...,100.0,4,3,3.31,1.62,1.74,0.98,4.2,3.15,0.21
9089,2021,2022-04-27,1818,UEFA Champions League,Liverpool,Villarreal,93.51,81.4,0.6634,0.1344,...,100.0,2,0,1.84,0.09,2.48,0.19,2.1,0.0,0.20
9377,2021,2022-05-03,1818,UEFA Champions League,Villarreal,Liverpool,79.46,93.92,0.2065,0.5609,...,100.0,2,3,1.48,1.59,0.86,1.61,2.1,3.15,0.23
9388,2021,2022-05-04,1818,UEFA Champions League,Real Madrid,Manchester City,85.82,93.69,0.2964,0.4737,...,100.0,3,1,2.6,1.73,2.4,1.84,3.15,1.05,0.23


# Images API

### Current Teams dict

In [6]:

team_dict = {'Manchester United': 'https://media.api-sports.io/football/teams/33.png',
             'Newcastle': 'https://media.api-sports.io/football/teams/34.png',
             'Bournemouth': 'https://media.api-sports.io/football/teams/35.png',
             'Watford': 'https://media.api-sports.io/football/teams/38.png',
             'Wolves': 'https://media.api-sports.io/football/teams/39.png',
             'Liverpool': 'https://media.api-sports.io/football/teams/40.png',
             'Southampton': 'https://media.api-sports.io/football/teams/41.png',
             'Arsenal': 'https://media.api-sports.io/football/teams/42.png',
             'Burnley': 'https://media.api-sports.io/football/teams/44.png',
             'Everton': 'https://media.api-sports.io/football/teams/45.png',
             'Leicester': 'https://media.api-sports.io/football/teams/46.png',
             'Tottenham Hotspur': 'https://media.api-sports.io/football/teams/47.png',
             'West Ham': 'https://media.api-sports.io/football/teams/48.png',
             'Chelsea': 'https://media.api-sports.io/football/teams/49.png',
             'Manchester City': 'https://media.api-sports.io/football/teams/50.png',
             'Brighton': 'https://media.api-sports.io/football/teams/51.png',
             'Crystal Palace': 'https://media.api-sports.io/football/teams/52.png',
             'Sheffield Utd': 'https://media.api-sports.io/football/teams/62.png',
             'Aston Villa': 'https://media.api-sports.io/football/teams/66.png',
             'Norwich': 'https://media.api-sports.io/football/teams/71.png',
             'Bayern Munich': 'https://media.api-sports.io/football/teams/157.png',
             'Fortuna Dusseldorf': 'https://media.api-sports.io/football/teams/158.png',
             'Hertha Berlin': 'https://media.api-sports.io/football/teams/159.png',
             'SC Freiburg': 'https://media.api-sports.io/football/teams/160.png',
             'VfL Wolfsburg': 'https://media.api-sports.io/football/teams/161.png',
             'Werder Bremen': 'https://media.api-sports.io/football/teams/162.png',
             'Borussia Monchengladbach': 'https://media.api-sports.io/football/teams/163.png',
             'FSV Mainz 05': 'https://media.api-sports.io/football/teams/164.png',
             'Borussia Dortmund': 'https://media.api-sports.io/football/teams/165.png',
             '1899 Hoffenheim': 'https://media.api-sports.io/football/teams/167.png',
             'Bayer Leverkusen': 'https://media.api-sports.io/football/teams/168.png',
             'Eintracht Frankfurt': 'https://media.api-sports.io/football/teams/169.png',
             'FC Augsburg': 'https://media.api-sports.io/football/teams/170.png',
             'RB Leipzig': 'https://media.api-sports.io/football/teams/173.png',
             'FC Schalke 04': 'https://media.api-sports.io/football/teams/174.png',
             'FC Heidenheim': 'https://media.api-sports.io/football/teams/180.png',
             'Union Berlin': 'https://media.api-sports.io/football/teams/182.png',
             'SC Paderborn 07': 'https://media.api-sports.io/football/teams/185.png',
             'FC Koln': 'https://media.api-sports.io/football/teams/192.png',
             'Barcelona': 'https://media.api-sports.io/football/teams/529.png',
             'Atletico Madrid': 'https://media.api-sports.io/football/teams/530.png',
             'Athletic Club': 'https://media.api-sports.io/football/teams/531.png',
             'Valencia': 'https://media.api-sports.io/football/teams/532.png',
             'Villarreal': 'https://media.api-sports.io/football/teams/533.png',
             'Sevilla': 'https://media.api-sports.io/football/teams/536.png',
             'Leganes': 'https://media.api-sports.io/football/teams/537.png',
             'Celta Vigo': 'https://media.api-sports.io/football/teams/538.png',
             'Levante': 'https://media.api-sports.io/football/teams/539.png',
             'Espanyol': 'https://media.api-sports.io/football/teams/540.png',
             'Real Madrid': 'https://media.api-sports.io/football/teams/541.png',
             'Alaves': 'https://media.api-sports.io/football/teams/542.png',
             'Real Betis': 'https://media.api-sports.io/football/teams/543.png',
             'Eibar': 'https://media.api-sports.io/football/teams/545.png',
             'Getafe': 'https://media.api-sports.io/football/teams/546.png',
             'Real Sociedad': 'https://media.api-sports.io/football/teams/548.png',
             'Granada CF': 'https://media.api-sports.io/football/teams/715.png',
             'Valladolid': 'https://media.api-sports.io/football/teams/720.png',
             'Osasuna': 'https://media.api-sports.io/football/teams/727.png',
             'Mallorca': 'https://media.api-sports.io/football/teams/798.png'}


### Adding Teams

In [9]:
f = open('./football_data/league_id.json', 'r')
league_ids = json.load(f)

In [7]:
for [id, season] in league_ids.values():
	url = "https://api-football-beta.p.rapidapi.com/teams"

	querystring = {"league":id,"season":season}

	headers = {
		"X-RapidAPI-Host": "api-football-beta.p.rapidapi.com",
		"X-RapidAPI-Key": "d78ca9f758msh31ad154b2fe50a8p12fbc9jsnabb8cabd4076"
	}

	response = requests.request("GET", url, headers=headers, params=querystring)
	teams = response.json()['response']
	for team in teams:
		team_dict[team['team']['name']] = team['team']['logo']

In [159]:
with open('./football_data/team_img.json', 'w') as f:
    json.dump(team_dict, f)

# Get Per League Standings

In [105]:
# # Create dict of id: [league name, season] 
# import requests

# url = "https://api-football-beta.p.rapidapi.com/leagues"

# headers = {
# 	"X-RapidAPI-Host": "api-football-beta.p.rapidapi.com",
# 	"X-RapidAPI-Key": "d78ca9f758msh31ad154b2fe50a8p12fbc9jsnabb8cabd4076"
# }

# response = requests.request("GET", url, headers=headers)

# league_id_dict = {}
# leagues = response.json()['response']
# for league in leagues:
#     league_id_dict[league['league']['name']] = [league['league']['id'], league['seasons'][-1]['year']]

# unique_leagues = list(rename_dict.keys())
# for league in list(league_id_dict.keys()):
#     if league not in unique_leagues:
#         del league_id_dict[league]

# with open('./football_data/league_id.json', 'w') as f:
#     json.dump(league_id_dict, f)

In [ ]:
f = open('./football_data/league_id.json', 'r')
league_ids = json.load(f)
len(league_ids)

In [ ]:
# Get standings by league id:
import requests

url = "https://api-football-beta.p.rapidapi.com/standings"

querystring = {"season":"2021","league":"39"}

headers = {
	"X-RapidAPI-Host": "api-football-beta.p.rapidapi.com",
	"X-RapidAPI-Key": "d78ca9f758msh31ad154b2fe50a8p12fbc9jsnabb8cabd4076"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)